In [1]:
from __future__ import division
import datetime
import matplotlib.pyplot as plt
import numpy as np
import v2gsim 
import utility
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
import os

# Create a project and initialize it with some itineraries
project = v2gsim.model.Project()
project = v2gsim.itinerary.from_excel(project, '../data/NHTS/California.xlsx')
project = v2gsim.itinerary.copy_append(project, nb_of_days_to_add=2)

# This function from the itinerary module return all the vehicles that
# start and end their day at the same location (e.g. home)
project.vehicles = v2gsim.itinerary.get_cycling_itineraries(project)

# Reduce the number of vehicles
project.vehicles = project.vehicles[0:100]

# Create some new charging infrastructures, append those new
# infrastructures to the project list of infrastructures
charging_stations = []
charging_stations.append(
    v2gsim.model.ChargingStation(name='L2', maximum_power=7200, minimum_power=0))
charging_stations.append(
    v2gsim.model.ChargingStation(name='L1_V1G', maximum_power=1400, minimum_power=0, post_simulation=True))
charging_stations.append(
    v2gsim.model.ChargingStation(name='L2_V2G', maximum_power=7200, minimum_power=-7200, post_simulation=True))
project.charging_stations.extend(charging_stations)

#Create a data frame with the new infrastructures mix and
# apply this mix at all the locations
df = pd.DataFrame(index=['L2', 'L1_V1G', 'L2_V2G'],
                      data={'charging_station': charging_stations,
                            'probability': [0.0, 0.4, 0.6]})
for location in project.locations:
    if location.category in ['Work', 'Home']:
        location.available_charging_station = df.copy()

# Initiate SOC and charging infrastructures
v2gsim.core.initialize_SOC(project, nb_iteration=2)

# Assign a basic result function to save power demand
for vehicle in project.vehicles:
    vehicle.result_function = v2gsim.post_simulation.netload_optimization.save_vehicle_state_for_optimization


# Launch the simulation
v2gsim.core.run(project, date_from=project.date + datetime.timedelta(days=1),
                date_to=project.date + datetime.timedelta(days=2),
                reset_charging_station=False)


# Look at the results
total_power_demand = v2gsim.post_simulation.result.total_power_demand(project)

# Optimization
myopti = v2gsim.post_simulation.netload_optimization.CentralOptimization(project, 10,
                                                                         project.date + datetime.timedelta(days=1),
                                                                         project.date + datetime.timedelta(days=2),
                                                                         minimum_SOC=0.1, maximum_SOC=0.95)

/Users/meiyewang/opt/anaconda3/envs/py27/lib/python2.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
/Users/meiyewang/opt/anaconda3/envs/py27/lib/python2.7/site-packages/v2gsim/core.py:194: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  axis=0)
core.initialize_SOC: 100%|####################################################|
core.run:  15%|#########                                                      |       mean  mean_rate       std  std_rate
0  0.950000   0.000000  0.000000  0.000000
1  0.926814   0.023186  0.110897 -0.110897
2  0.926647   0.000167  0.110849  0.000049

core.run: 100%|###############################################################|core.run: 100%|####

In [3]:
import numpy as np
import v2gsim 
import utility
from pyomo.opt import SolverFactory
from pyomo.core import Var
from pyomo.core import Param
from operator import itemgetter
import pandas as pd
import os

myresult=pd.read_csv('../data/vehicle/VehiclesCap_iteration1.csv',header=0)
N = 6# from 10mins to 1 hour
myresult=myresult.groupby(myresult.index // N).sum()/6
myresult=myresult.rename(columns={'EnergyDemand':'netload','EnergyGeneration':'gen_capacity_veh','Regup':'regup_capacity_veh','Regdown':'regdown_capacity_veh'})
myresult=pd.concat([myresult,myresult],axis=0)
myresult.index=range(myresult.shape[0])
df_veh_cap=myresult[['gen_capacity_veh'	,'regup_capacity_veh',	'regdown_capacity_veh']]

df_load = pd.read_csv('../data/netload/20200801CAISO.csv',header=0)[:48]
print(df_load)
df_load['netload']=myresult['netload']+df_load['netload']
print(df_veh_cap)


date_time  netload
0   8/1/20 00:00  26938.0
1   8/1/20 01:00  25315.0
2   8/1/20 02:00  24354.0
3   8/1/20 03:00  23649.0
4   8/1/20 04:00  23324.0
5   8/1/20 05:00  23283.0
6   8/1/20 06:00  23272.0
7   8/1/20 07:00  23928.0
8   8/1/20 08:00  24504.0
9   8/1/20 09:00  25297.0
10  8/1/20 10:00  26265.0
11  8/1/20 11:00  27735.0
12  8/1/20 12:00  29391.0
13  8/1/20 13:00  31113.0
14  8/1/20 14:00  32720.0
15  8/1/20 15:00  34347.0
16  8/1/20 16:00  35803.0
17  8/1/20 17:00  37004.0
18  8/1/20 18:00  37145.0
19  8/1/20 19:00  36088.0
20  8/1/20 20:00  34735.0
21  8/1/20 21:00  32750.0
22  8/1/20 22:00  30083.0
23  8/1/20 23:00  27591.0
24  8/2/20 00:00  25614.0
25  8/2/20 01:00  24108.0
26  8/2/20 02:00  22996.0
27  8/2/20 03:00  22391.0
28  8/2/20 04:00  22074.0
29  8/2/20 05:00  21908.0
30  8/2/20 06:00  21676.0
31  8/2/20 07:00  21980.0
32  8/2/20 08:00  22366.0
33  8/2/20 09:00  23029.0
34  8/2/20 10:00  24042.0
35  8/2/20 11:00  25653.0
36  8/2/20 12:00  27439.0
37  8/2/20 13:00  2

In [10]:
import pandas as pd
import datetime 
data_name = 'iteration1_correct'
# # Load the load data
load = pd.read_csv('../data/netload/20200801CAISO.csv',header=0)
print(load[:25].shape)
#CAISO1819: netload in the unit of MW
startday = datetime.datetime(2020, 8, 1)#iterate through 1 day per time

load['date_time'] = [datetime.datetime.strptime(x,"%m/%d/%y %H:%M") for x in load['date_time']]
load=load.set_index(load['date_time'])
load=load.drop(columns='date_time')

# ####################################### add the component of loop around day
load = pd.DataFrame(load[startday: startday +datetime.timedelta(days=1)]['netload'])

#iteration1
pr=pd.read_csv('../data/price/price'+data_name+'.csv',header=0)
print(load[:25].shape)
print(pr[:25].shape)

# pr=pr[:25].set_index(load[:25].index)

(25, 2)
(25, 1)
(24, 4)


In [7]:
import pandas as pd
import datetime 
data_name = 'iteration1_correct'
mwh_pd=pd.read_csv('../data/generator/GeneratorResult'+data_name+'.csv')
mwh_pd_gr=mwh_pd.groupby(mwh_pd['Hour'])['Generation','Reg Up','Reg Down'].sum()


myresult=pd.read_csv('../data/vehicle/VehiclesCap_iteration1.csv',header=0)
N = 6# from 10mins to 1 hour
myresult=myresult.groupby(myresult.index // N).sum()/6
myresult=myresult.rename(columns={'EnergyDemand':'netload','EnergyGeneration':'gen_capacity_veh','Regup':'regup_capacity_veh','Regdown':'regdown_capacity_veh'})

df_veh_cap=myresult[['gen_capacity_veh'	,'regup_capacity_veh',	'regdown_capacity_veh']]
df_load = pd.read_csv('../data/netload/20200801CAISO.csv',header=0)[:26]
mwh_pd_gr['load']=myresult['netload']+df_load['netload']
mwh_pd_gr['Generation/Load']=mwh_pd_gr['Generation']/mwh_pd_gr['load']
mwh_pd_gr['Regup/Load']=mwh_pd_gr['Reg Up']/mwh_pd_gr['load']
mwh_pd_gr['RegDown/Load']=mwh_pd_gr['Reg Down']/mwh_pd_gr['load']
mwh_pd_gr

,Generation,Reg Up,Reg Down,load,Generation/Load,Regup/Load,RegDown/Load
Hour,,,,,,,
1,304804.149333,1724.032,1939.536000,25315.0,12.040456,0.068103,0.076616
2,181626.226667,1620.160,255333.333333,24354.0,7.457758,0.066525,10.484246
3,235041.661333,1558.656,1753.488000,23649.0,9.938757,0.065908,0.074146
4,230138.653333,1513.536,206666.666667,23324.0,9.867032,0.064892,8.860687
5,265096.661333,1492.736,1679.328000,23283.0,11.385846,0.064113,0.072127
6,185349.266667,1490.112,237266.666667,23272.0,7.964475,0.064030,10.195371
7,201569.954000,1489.408,1675.584000,23928.0,8.424020,0.062245,0.070026
8,186104.236667,1531.392,193080.000000,24504.0,7.594851,0.062496,7.879530
9,208279.521333,1568.256,1764.288000,25297.0,8.233368,0.061994,0.069743


In [12]:
mwh_pd_gr_s=mwh_pd[mwh_pd['Generator']=='solar']

mwh_pd_gr_s

,Generator,Day,Hour,Generation,Reg Up,Reg Down
797,solar,1,1,0.00,0.0,0.0
1597,solar,1,2,0.00,0.0,0.0
2397,solar,1,3,0.00,0.0,0.0
3197,solar,1,4,0.00,0.0,0.0
3997,solar,1,5,0.00,0.0,0.0
4797,solar,1,6,0.00,0.0,0.0
5597,solar,1,7,15.37,0.0,0.0
6397,solar,1,8,15.37,0.0,0.0
7197,solar,1,9,0.00,0.0,0.0
7997,solar,1,10,0.00,0.0,0.0


In [13]:

mwh_pd_gr_w=mwh_pd[mwh_pd['Generator']=='wind']

mwh_pd_gr_w

,Generator,Day,Hour,Generation,Reg Up,Reg Down
799,wind,1,1,200.48000,0.0,0.0
1599,wind,1,2,150.36000,0.0,0.0
2399,wind,1,3,100.24000,0.0,0.0
3199,wind,1,4,50.12000,0.0,0.0
3999,wind,1,5,0.00000,0.0,0.0
4799,wind,1,6,0.00000,0.0,0.0
5599,wind,1,7,0.00000,0.0,0.0
6399,wind,1,8,0.00000,0.0,0.0
7199,wind,1,9,0.00000,0.0,0.0
7999,wind,1,10,0.00000,0.0,0.0


In [18]:
myresult=pd.read_csv('../data/vehicle/VehiclesCap_iteration1.csv',header=0)
N = 6# from 10mins to 1 hour
myresult=myresult.groupby(myresult.index // N).sum()/6
myresult=myresult.rename(columns={'EnergyDemand':'netload','EnergyGeneration':'gen_capacity_veh','Regup':'regup_capacity_veh','Regdown':'regdown_capacity_veh'})
myresult=pd.concat([myresult,myresult],axis=0)
myresult.index=range(myresult.shape[0])
df_veh_cap=myresult[['gen_capacity_veh'	,'regup_capacity_veh',	'regdown_capacity_veh']]
print(df_veh_cap)
with open(''+str('vehicle_test')+'.dat', 'w') as f:
    f.write('param:' + '\t')
    for c in df_veh_cap.columns:
        if c != 'name':
            f.write(c + '\t')
    f.write(':=\n\n')
    for i in range(0,len(df_veh_cap)):    
        for c in df_veh_cap.columns: 
            if c=='gen_capacity_veh':
                f.write(str(i+1) + '\t' + str(df_veh_cap.loc[i,c]) + '\t') 
            else:
                f.write(str(df_veh_cap.loc[i,c]) + '\t')            
        f.write('\n')
    f.write(';\n\n')


gen_capacity_veh  regup_capacity_veh  regdown_capacity_veh
0                0.0       135166.666667         300833.333333
1                0.0       180666.666667         255333.333333
2                0.0       204466.666667         231533.333333
3                0.0       229333.333333         206666.666667
4                0.0       172766.666667         261833.333333
5                0.0       184606.666667         237266.666667
6                0.0       207080.000000         198300.000000
7                0.0       185326.666667         193080.000000
8                0.0       174060.000000         204853.333333
9                0.0       172133.333333         200946.666667
10               0.0       170266.666667         200933.333333
11               0.0       192093.333333         172726.666667
12               0.0       184340.000000         185953.333333
13               0.0       190320.000000         186357.361663
14               0.0       251220.000000         100800.000

In [34]:
import pandas as pd 
myresult=pd.read_csv('../data/vehicle/VehiclesCap_iteration1.csv',header=0)
N = 6
myresult=myresult.groupby(myresult.index // N).sum()/6
myresult=myresult.rename(columns={'EnergyDemand':'netload','EnergyGeneration':'gen_capacity_veh','Regup':'regup_capacity_veh','Regdown':'regdown_capacity_veh'})

df_load = pd.read_csv('../data/netload/CAISO1819.csv',header=0)[:24]
df_load['netload']=myresult['netload']+df_load['netload']

myresult

,netload,gen_capacity_veh,regup_capacity_veh,regdown_capacity_veh
0,0.0,0.0,135166.666667,300833.333333
1,0.0,0.0,180666.666667,255333.333333
2,0.0,0.0,204466.666667,231533.333333
3,0.0,0.0,229333.333333,206666.666667
4,0.0,0.0,172766.666667,261833.333333
5,0.0,0.0,184606.666667,237266.666667
6,0.0,0.0,207080.000000,198300.000000
7,0.0,0.0,185326.666667,193080.000000
8,0.0,0.0,174060.000000,204853.333333
9,0.0,0.0,172133.333333,200946.666667


In [32]:
myresult[['gen_capacity_veh'	,'regup_capacity_veh',	'regdown_capacity_veh']]

,gen_capacity_veh,regup_capacity_veh,regdown_capacity_veh
0,0.0,88400.0,347600.0
1,0.0,252000.0,184000.0
2,0.0,236400.0,199600.0
3,0.0,35200.0,400800.0
4,0.0,46000.0,390000.0
5,0.0,153000.0,283000.0
6,0.0,154400.0,281600.0
7,0.0,234000.0,202000.0
8,0.0,239800.0,196200.0
9,0.0,220000.0,216000.0


In [19]:
# # Load the load data
load=pd.read_csv('../data/netload/CAISO1819.csv')
#CAISO1819: netload in the unit of W
startday = datetime.datetime(2018, 7, 2)#iterate through 1 day per time

load['date_time'] = [datetime.datetime.strptime(x,"%m/%d/%Y %H:%M") for x in load['date_time']]
load=load.set_index(load['date_time'])
load=load.drop(columns='date_time')

# ####################################### add the component of loop around day
load = pd.DataFrame(load[startday: startday +datetime.timedelta(days=1)]['netload'])


price=pd.read_csv("../data/price/price.csv")
price=price[:24].set_index(load[:24].index)



In [25]:

pr_e.update(temp_price.to_dict()['pr_e'])


NameError: name 'pr_e' is not defined

In [8]:
load=load.resample(str(10) + 'T').first()
load=load.fillna(method='ffill')
load

,netload
date_time,
2018-07-02 00:00:00,31187.0
2018-07-02 00:10:00,31187.0
2018-07-02 00:20:00,31187.0
2018-07-02 00:30:00,31187.0
2018-07-02 00:40:00,31187.0
2018-07-02 00:50:00,31187.0
2018-07-02 01:00:00,32303.0
2018-07-02 01:10:00,32303.0
2018-07-02 01:20:00,32303.0


TypeError: Index does not support mutable operations

In [17]:
# # Load the load data
load=pd.read_csv('../data/netload/CAISO1819.csv')
startday = datetime.datetime(2018, 7, 2)#iterate through 1 day per time

load['date_time'] = [datetime.datetime.strptime(x,"%m/%d/%Y %H:%M") for x in load['date_time']]
load=load.set_index(load['date_time'])
load=load.drop(columns='date_time')

# ####################################### add the component of loop around day
load = pd.DataFrame(load[startday: startday +datetime.timedelta(days=1)]['demand (MW)'])

# load = pd.DataFrame(load[startday]['demand (MW)'])
# load

,demand (MW)
date_time,
2018-07-02 00:00:00,31187
2018-07-02 01:00:00,32303
2018-07-02 02:00:00,32870
2018-07-02 03:00:00,32478
2018-07-02 04:00:00,31983
2018-07-02 05:00:00,30949
2018-07-02 06:00:00,28655
2018-07-02 07:00:00,26236
2018-07-02 08:00:00,24304


In [16]:
price_pd=getprice(instance,cost_pd)

myresult = myopti.solve(project, load * 1000000,1500000, price=price_pd,
                    peak_shaving='economic', SOC_margin=0.05)

# Get the result in the right format
temp_vehicle = pd.DataFrame(
    (total_power_demand['total'] - myresult['vehicle_before'] + myresult['vehicle_after']) *
    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle = temp_vehicle.rename(columns={0: 'vehicle'})
temp_vehicle['index'] = range(0, len(temp_vehicle))
temp_vehicle = temp_vehicle.set_index(['index'], drop=True)

temp_load = load.copy()
temp_load = temp_load.resample('60S')
temp_load = temp_load.fillna (method='ffill').fillna(method='bfill')
temp_load = temp_load.head(len(temp_vehicle))
tempIndex = temp_load.index
temp_load['index'] = range(0, len(temp_vehicle))
temp_load = temp_load.set_index(['index'], drop=True)

temp_result = pd.DataFrame(temp_load['demand (MW)'] + temp_vehicle['vehicle'])
temp_result = temp_result.rename(columns={0: 'demand (MW)'})
temp_result = temp_result.set_index(tempIndex)
temp_load = temp_load.set_index(tempIndex)
temp_vehicle = temp_vehicle.set_index(tempIndex)

temp_vehicle_demand = pd.DataFrame(
    (myresult['vehicle_after_demand']) *    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle_demand = temp_vehicle_demand.set_index(tempIndex)

temp_vehicle_generation = pd.DataFrame(
    (-myresult['vehicle_after_generation']) *    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle_generation = temp_vehicle_generation.set_index(tempIndex)

temp_vehicle_regdown = pd.DataFrame(
    (myresult['vehicle_after_regdown']) *    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle_demand = temp_vehicle_demand.set_index(tempIndex)

temp_vehicle_regup = pd.DataFrame(
    (-myresult['vehicle_after_regup']) *    (1500000 / len(project.vehicles)) / (1000 * 1000))  # Scale up and W to MW
temp_vehicle_generation = temp_vehicle_generation.set_index(tempIndex)


plt.plot(temp_load['demand (MW)'], label='load')
plt.plot(temp_result['demand (MW)'], label='load + vehicles')
plt.ylabel('Power (MW)')
plt.legend()
plt.show()

NameError: name 'getprice' is not defined